In [9]:
import pandas as pd
import re
import sys

In [10]:
citations_df = pd.read_csv('../data-output/citation_content_all_batches.csv')
plants_df = pd.read_csv('../data-output/FMFO Plants in Latin America.csv')

In [11]:
def clean_company_name(name):
    """
    Clean company name by removing legal entity suffixes and standardizing format.
    
    Args:
        name (str): Company name to clean
        
    Returns:
        str: Cleaned company name
    """
    if pd.isna(name) or not name:
        return ""
    
    # Convert to string and lowercase
    name = str(name).lower().strip()
    
    # Remove common legal entity suffixes
    suffixes = [
        r'\bs\.a\.$', r'\bs\.a\b', 
        r'\bs\.a\.c\.$', r'\bs\.a\.c\b',
        r'\bs\.a\. de c\.v\.$', r'\bs\.a\. de c\.v\b',
        r'\bs\.r\.l\.$', r'\bs\.r\.l\b',
        r'\bltda\.$', r'\bltda\b',
        r'\bs\.a\.p\.i\. de c\.v\.$', r'\bs\.a\.p\.i\. de c\.v\b',
        r'\bcia\. ltda\.$', r'\bcia\. ltda\b',
        r'\bs\. de r\.l\. de c\.v\.$', r'\bs\. de r\.l\. de c\.v\b',
        r'\bsa de cv$', r'\bsa de cv\b'
    ]
    
    for suffix in suffixes:
        name = re.sub(suffix, '', name)
    
    # Remove parentheses and their contents
    name = re.sub(r'\([^)]*\)', '', name)
    
    # Remove quotes
    name = name.replace('"', '').replace("'", '')
    
    # Remove leading/trailing whitespace and commas
    name = name.strip(', ')
    
    return name

In [12]:
# Clean plant names for matching
plants_df['clean_name'] = plants_df['Company name'].apply(clean_company_name)

# Initialize a column for storing matching URLs if it doesn't exist
if 'Crime Report Links' not in plants_df.columns:
    plants_df['Crime Report Links'] = ''

# Function to search for plant names in citation content
def find_plant_matches(row, plants_df):
    content = str(row['content']).lower()
    url = row['url']
    
    for idx, plant_row in plants_df.iterrows():
        clean_name = plant_row['clean_name']
        
        # Skip empty names
        if not clean_name:
            continue
            
        # Check if plant name appears in the content
        if clean_name in content:
            # If the cell already has content, append to it
            current_links = str(plant_row['Crime Report Links']) if not pd.isna(plant_row['Crime Report Links']) else ''
            if current_links and current_links.lower() != 'nan':
                # Check if URL is already in the links
                if url not in current_links:
                    plants_df.at[idx, 'Crime Report Links'] = current_links + ', ' + url
            else:
                plants_df.at[idx, 'Crime Report Links'] = url
    
    return plants_df

# Process each citation
print("Searching for plant names in citation content...")
for idx, citation_row in citations_df.iterrows():
    plants_df = find_plant_matches(citation_row, plants_df)

# Display summary
total_plants = len(plants_df)
plants_with_links = len(plants_df[plants_df['Crime Report Links'].str.strip() != ''])
print(f"Summary: Found matching content for {plants_with_links} out of {total_plants} plants.")

# Drop the temporary clean_name column
plants_df = plants_df.drop(columns=['clean_name'])

# Display the first few rows with matches
plants_df[plants_df['Crime Report Links'].str.strip() != ''].head()


Searching for plant names in citation content...
Summary: Found matching content for 219 out of 219 plants.


,Company name,Country,Crime report links,Crime Report Links
0,Mundo Branco S. A.,Argentina,NaN,NaN
1,(Quirola),Ecuador,NaN,NaN
2,San Arawa S.A.,Argentina,NaN,NaN
3,Agustiner S.A.,Argentina,NaN,NaN
4,Coomarpes Ltda.,Argentina,NaN,NaN


In [16]:
# Display all rows where either 'Crime report links' or 'Crime Report Links' has a value
mask = (
    (~pd.isna(plants_df['Crime report links']) & (plants_df['Crime report links'] != '')) | 
    (~pd.isna(plants_df['Crime Report Links']) & (plants_df['Crime Report Links'] != ''))
)

# Show the filtered dataframe
plants_df[mask]


,Company name,Country,Crime report links,Crime Report Links
34,Orizon S.A.,Chile,NaN,"https://goo.su/f3FzQs, https://goo.su/GRDzA, h..."
52,FASA,Brazil,NaN,https://goo.su/zQRPx1
96,Unknown,Peru,"https://goo.su/IGDZ, https://goo.su/zQk0Cug","https://goo.su/IGDZ, https://goo.su/zQk0Cug"
119,Pesquera Capricornio S.A.,Peru,NaN,https://goo.su/RspfM90
134,TASA,Peru,"https://goo.su/zIqY, https://goo.su/g1yUfGG, h...","https://goo.su/zIqY, https://goo.su/g1yUfGG, h..."
151,San Fernando S.A.,Peru,NaN,"https://goo.su/UIPwu, https://goo.su/4jcmR, ht..."
218,Urisa S.A.,Ecuador,NaN,https://goo.su/iOkUexI


In [19]:
# Combine 'Crime report links' and 'Crime Report Links' columns
print("Combining URL columns and removing duplicates...")

# Function to combine URLs without duplicates
def combine_urls(row):
    urls = []
    
    # Get URLs from both columns
    for col in ['Crime report links', 'Crime Report Links']:
        if not pd.isna(row[col]) and row[col] != '':
            # Split by comma and strip whitespace
            col_urls = [url.strip() for url in str(row[col]).split(',')]
            urls.extend(col_urls)
    
    # Remove duplicates while preserving order
    unique_urls = []
    for url in urls:
        if url not in unique_urls and url.lower() != 'nan':
            unique_urls.append(url)
    
    # Join URLs with comma and space
    return ', '.join(unique_urls) if unique_urls else ''

# Apply the function to combine URLs
plants_df['Crime Report Links'] = plants_df.apply(combine_urls, axis=1)

# Drop the 'Crime report links' column
plants_df = plants_df.drop(columns=['Crime report links'])

# Save the updated dataframe to CSV
output_path = '../data-output/FMFO_Plants_Updated.csv'
plants_df.to_csv(output_path, index=False)

print(f"Combined URL columns and saved to {output_path}")

# Display the first few rows with links
plants_df[plants_df['Crime Report Links'] != ''].head()

Combining URL columns and removing duplicates...
Combined URL columns and saved to ../data-output/FMFO_Plants_Updated.csv


,Company name,Country,Crime Report Links
34,Orizon S.A.,Chile,"https://goo.su/f3FzQs, https://goo.su/GRDzA, h..."
52,FASA,Brazil,https://goo.su/zQRPx1
96,Unknown,Peru,"https://goo.su/IGDZ, https://goo.su/zQk0Cug"
119,Pesquera Capricornio S.A.,Peru,https://goo.su/RspfM90
134,TASA,Peru,"https://goo.su/zIqY, https://goo.su/g1yUfGG, h..."
